In [ ]:
 ======================================================
# 📦 Scarica i ticker di S&P 500 e Nasdaq 100 da Wikipedia
# ======================================================
 
!pip install yfinance openpyxl pandas requests matplotlib beautifulsoup4 lxml --quiet
 
import pandas as pd
import requests
import yfinance as yf
import matplotlib.pyplot as plt
import warnings
from io import StringIO
 
warnings.filterwarnings("ignore", category=FutureWarning)
 
print("📥 Scaricamento ticker S&P 500 e Nasdaq 100 da Wikipedia...")
 
tickers = set()
 
# Funzione helper per leggere una tabella da Wikipedia con header browser-like
def read_html_with_headers(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0 Safari/537.36"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return pd.read_html(StringIO(response.text))
 
# ======================================================
# 🧾 S&P 500
# ======================================================
try:
    url_sp500 = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = read_html_with_headers(url_sp500)
    df_sp500 = tables[0]  # La prima tabella è quella giusta
    sp500_tickers = df_sp500["Symbol"].str.replace(".", "-", regex=False).tolist()
    tickers.update(sp500_tickers)
    print(f"✅ Ticker S&P 500 scaricati: {len(sp500_tickers)}")
except Exception as e:
    print("❌ Errore caricamento S&P 500:", e)
 
# ======================================================
# 🧾 Nasdaq 100
# ======================================================
try:
    url_nasdaq100 = "https://en.wikipedia.org/wiki/NASDAQ-100"
    tables = read_html_with_headers(url_nasdaq100)
    # Cerca la tabella corretta
    df_nasdaq = next(df for df in tables if any(col in df.columns for col in ["Ticker", "Symbol"]))
    ticker_col = "Ticker" if "Ticker" in df_nasdaq.columns else "Symbol"
    nasdaq_tickers = df_nasdaq[ticker_col].str.replace(".", "-", regex=False).tolist()
    tickers.update(nasdaq_tickers)
    print(f"✅ Ticker Nasdaq 100 scaricati: {len(nasdaq_tickers)}")
except Exception as e:
    print("❌ Errore caricamento Nasdaq 100:", e)
 
# ======================================================
# 📊 Risultati combinati
# ======================================================
all_tickers = sorted(tickers)
 
print(f"\n📈 Totale ticker unici raccolti: {len(all_tickers)}")
print("Esempio dei primi 20 ticker:", all_tickers[:20])
 
# Salva in variabile per la cella successiva
cleaned_combined_tickers = all_tickers
 
# ======================================================
# ℹ️ Nota:
# La variabile `cleaned_combined_tickers` contiene tutti i ticker unificati
# Potrai ora usarla nella prossima cella per l’analisi (es. % sopra SMA200)
# ======================================================
 

In [ ]:
======================================================
# 📦 Scarica dati e calcola indicatore (% sopra MA200)
# ======================================================
 
# Assicurati che cleaned_combined_tickers sia disponibile dalla prima cella
if 'cleaned_combined_tickers' not in locals():
    raise RuntimeError("Variabile 'cleaned_combined_tickers' non trovata. Esegui prima la prima cella.")
 
start = '2010-01-01'        # Data di inizio per il download dei dati
 
# Definisci gli indici da analizzare e i loro ticker corrispondenti
indices_to_analyze = {
    "S&P 500": "SPY",  # SPY è un ETF che replica l'S&P 500
    "Nasdaq 100": "QQQ" # QQQ è un ETF che replica il Nasdaq 100
}
 
# Dictionaries to store data and indicators for each index
data_dict = {}
index_dict = {}
above200_dict = {}
valid_tickers_dict = {}
 
print(f"\n{'='*50}")
print("📥 Scaricamento dati e calcolo indicatore per Indici")
print(f"{'='*50}")
 
for index_name, index_ticker in indices_to_analyze.items():
    print(f"\nElaborazione per {index_name} ({index_ticker})...")
 
    # Use all cleaned_combined_tickers for analysis
    tickers = cleaned_combined_tickers
 
    if not tickers:
        print(f"❌ Nessun ticker trovato per {index_name}.")
        continue # Skip to the next index
 
    # DOWNLOAD DATI
    try:
        # Download data for all tickers at once
        data = yf.download(tickers, start=start, period="max", interval="1d", progress=False, auto_adjust=False)['Adj Close']
    except Exception as e:
        print(f"❌ Errore durante il download dei dati dei ticker per {index_name}: {e}")
        continue # Skip to the next index
 
    if data.empty:
        print(f"❌ Nessun dato valido scaricato per i ticker di {index_name}.")
        continue # Skip to the next index
 
    # Download index data
    try:
        index = yf.download(index_ticker, start=start, period="max", interval="1d", progress=False, auto_adjust=False)['Adj Close']
    except Exception as e:
        print(f"❌ Errore durante il download dei dati dell'indice {index_ticker}: {e}")
        continue # Skip to the next index
 
    if index.empty:
        print(f"❌ Nessun dato valido scaricato per l'indice {index_ticker}.")
        continue # Skip to the next index
 
    # Align data and index
    data, index = data.align(index, join='inner', axis=0)
 
    # Filter out tickers that have become all NaNs after alignment
    data = data.dropna(axis=1, how='all')
    valid_tickers = data.columns.tolist()
 
    if not valid_tickers:
        print(f"❌ Nessun ticker valido rimasto dopo l'allineamento dei dati per {index_name}.")
        continue # Skip to the next index
 
    print(f"\n✅ Dati scaricati per {len(valid_tickers)} tickers validi per {index_name}.")
 
    # ==========================================
    # CALCOLO INDICATORE (% sopra MA200)
    # ==========================================
    ma200 = data.rolling(200).mean()
    # Use the length of valid_tickers AFTER alignment for the calculation
    above200 = (data > ma200).sum(axis=1) / len(valid_tickers) * 100
 
    # Store data and indicator in dictionaries
    data_dict[index_name] = data
    index_dict[index_name] = index
    above200_dict[index_name] = above200
    valid_tickers_dict[index_name] = valid_tickers
 
print(f"\n{'='*50}")
print("✅ Scaricamento dati e calcolo indicatore completati.")
print("Le variabili 'data_dict', 'index_dict', 'above200_dict', 'valid_tickers_dict' contengono i dati per ciascun indice.")

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns # Import seaborn for heatmap
 
# ==========================================
# PARAMETRI MODIFICABILI (per backtest variabile)
# ==========================================
 
# Inserisci qui i parametri che vuoi testare manualmente
fixed_parameters = {
    "S&P 500": {"buy_threshold": 21, "holding_days": 110},  # Esempio: puoi cambiare questi valori
    "Nasdaq 100": {"buy_threshold": 31, "holding_days": 70} # Esempio: puoi cambiare questi valori
}
 
# ==========================================
# ANALISI PER OGNI INDICE (S&P 500 e Nasdaq 100)
# ==========================================
 
# Assicurati che i dati siano disponibili dalla cella precedente (Scarica dati)
if 'above200_dict' not in locals() or 'index_dict' not in locals():
    raise RuntimeError("Variabili necessarie ('above200_dict' o 'index_dict') non trovate. Esegui prima la cella 'Scarica dati'.")
 
# Definisci gli indici da analizzare (basandosi sulle chiavi dei dictionary di dati)
indices_to_analyze = above200_dict.keys()
 
# Loop through each index
for index_name in indices_to_analyze:
 
    print(f"\n{'='*50}")
    print(f"🚀 Analisi Backtesting con Parametri Variabili per {index_name}")
    print(f"{'='*50}")
 
    # Get the fixed parameters for the current index
    if index_name not in fixed_parameters:
        print(f"❌ Parametri fissi non definiti per {index_name}. Salto l'analisi.")
        continue
 
    buy_threshold = fixed_parameters[index_name]["buy_threshold"]
    holding_days = fixed_parameters[index_name]["holding_days"]
 
    print(f"Utilizzo parametri: Soglia BUY={buy_threshold}%, Holding Days={holding_days} giorni")
 
    # Get data for the current index from the dictionaries
    above200 = above200_dict[index_name]
    index = index_dict[index_name]
 
    # ==========================================
    # FUNZIONE DI BACKTEST (uscita dopo X giorni - entrata su cross)
    # ==========================================
    # Ensure the function is defined here or accessible globally if defined elsewhere
    def backtest_timed_exit(above200, index, buy_thr, hold_days):
        position = 0
        days_in_trade = 0
        signal = pd.Series(0, index=above200.index)
 
        for i in range(1, len(signal)):
            # Check if the current date and previous date exist in above200 and index
            if (signal.index[i] in above200.index and signal.index[i] in index.index and
                signal.index[i-1] in above200.index and signal.index[i-1] in index.index): # Ensure previous day also exists in index and above200
 
                # Entry condition: only if flat AND indicator crosses below buy_thr
                if position == 0 and above200.iloc[i] < buy_thr and above200.iloc[i-1] >= buy_thr:
                    position = 1
                    days_in_trade = 0 # Reset days_in_trade on new entry
 
                elif position == 1:
                    days_in_trade += 1
                    if days_in_trade >= hold_days:
                        position = 0 # Exit after hold_days
 
                signal.iloc[i] = position
 
        returns = index.pct_change().fillna(0)
        # Ensure signal is aligned to the returns index before multiplication
        aligned_signal = signal.reindex(returns.index).fillna(0)
        # Calculate strategy returns by multiplying index returns with the aligned signal
        # Ensure returns is a Series before multiplication if it was a DataFrame
        if isinstance(returns, pd.DataFrame):
            strategy_returns = returns.iloc[:, 0] * aligned_signal
        else: # It's already a Series
            strategy_returns = returns * aligned_signal
 
        # Ensure strategy is a Series before cumsum
        strategy = strategy_returns.cumsum()
 
        total_return = np.exp(strategy.iloc[-1]) - 1
        cagr = (1 + total_return) ** (252 / len(index)) - 1 # Using len(index) for days in CAGR calculation
 
        return cagr, strategy, signal
 
    # ==========================================
    # ESECUZIONE BACKTEST CON PARAMETRI VARIABILI
    # ==========================================
    print(f"\n🔬 Esecuzione backtest con parametri variabili per {index_name}...")
 
    try:
        cagr, strat, signal = backtest_timed_exit(above200, index, buy_threshold, holding_days)
 
        print(f"\n✅ Backtest completato per {index_name}:")
        print(f"   Parametri utilizzati: Soglia BUY={buy_threshold}%, Holding Days={holding_days} giorni")
        print(f"   CAGR: {cagr:.4f}")
 
        # ==========================================
        # GRAFICO EQUITY LINE
        # ==========================================
        plt.figure(figsize=(12, 6)) # Increased figure size slightly
        # Ensure plotting data is aligned
        aligned_strat = np.exp(strat).reindex(index.index).ffill() # Use .ffill()
        aligned_index = (index / index.iloc[0]).reindex(index.index).ffill() # Use .ffill()
 
        plt.plot(aligned_strat, label=f'Strategy (Buy<{buy_threshold}%, hold {holding_days}d, cross)')
        plt.plot(aligned_index, label='Buy & Hold', alpha=0.5)
        plt.title(f"Equity line strategia breadth su {index_name} (holding {holding_days} giorno/i, entrata su cross)")
        plt.xlabel("Date") # Added X-axis label
        plt.ylabel("Cumulative Return (Factor)") # Added Y-axis label
        plt.legend()
        plt.grid(True)
        plt.xticks(rotation=45) # Rotate x-axis labels
        plt.tight_layout() # Adjust layout to prevent labels overlapping
        plt.show()
 
        # ==========================================
        # GRAFICO DELL’INDICATORE + SEGNALI
        # ==========================================
        plt.figure(figsize=(12, 6)) # Increased figure size slightly
        plt.plot(above200, label='% sopra MA200')
        plt.axhline(buy_threshold, color='red', linestyle='--', label=f'Soglia Buy {buy_threshold}%')
        # Ensure signal is aligned for plotting
        aligned_signal = signal.reindex(above200.index).fillna(0)
        plt.fill_between(above200.index, 0, 100, where=aligned_signal > 0, color='green', alpha=0.1, label='Posizione Long')
        plt.title(f"Indicatore breadth con segnali di acquisto su {index_name} (holding {holding_days} giorno/i, entrata su cross)")
        plt.xlabel("Date") # Added X-axis label
        plt.ylabel("% Above MA200") # Added Y-axis label
        plt.legend()
        plt.grid(True)
        plt.xticks(rotation=45) # Rotate x-axis labels
        plt.tight_layout() # Adjust layout
        plt.show()
 
    except Exception as e:
        print(f"❌ Errore durante il backtest con parametri variabili per {index_name}: {e}")
 
# ==========================================
# RIEPILOGO FINALE BACKTEST VARIABILE
# ==========================================
print(f"\n{'='*50}")
print("📊 Riepilogo Backtest con Parametri Variabili")
print(f"{'='*50}")
 
# Note: The results for the fixed backtest are printed directly above for each index.
print("Consulta l'output sopra per i risultati dettagliati del backtest con parametri variabili per ciascun indice.")


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns # Import seaborn for heatmap
 
# ==========================================
# OTTIMIZZAZIONE PARAMETRI (Soglia BUY e Holding Days)
# ==========================================
 
# Assicurati che i dati siano disponibili dalla cella precedente (Scarica dati)
if 'above200_dict' not in locals() or 'index_dict' not in locals():
    raise RuntimeError("Variabili necessarie ('above200_dict' o 'index_dict') non trovate. Esegui prima la cella 'Scarica dati'.")
 
buy_threshold_range = range(5, 55, 2) # Range e passo per testare le soglie BUY
holding_days_range = range(20, 120, 10) # Range e passo per testare i giorni di holding
 
optimization_results = {} # Dictionary to store optimization results for heatmap
best_results_optimization = {} # Dictionary to store best results from optimization
 
print(f"\n{'='*50}")
print("🔬 Esecuzione Ottimizzazione Parametri (Soglia BUY vs Holding Days)")
print(f"{'='*50}")
 
# Definisci gli indici da analizzare (basandosi sulle chiavi dei dictionary di dati)
indices_to_analyze = above200_dict.keys()
 
for index_name in indices_to_analyze:
    print(f"\nOttimizzazione per {index_name}...")
 
    # Get data for the current index from the dictionaries
    above200_opt = above200_dict[index_name]
    index_opt = index_dict[index_name]
 
    results_grid = pd.DataFrame(index=list(buy_threshold_range), columns=list(holding_days_range))
 
    for buy in buy_threshold_range:
        for hold in holding_days_range:
            try:
                # Use data specific to this optimization process
                # Ensure backtest_timed_exit function is accessible (defined in variable backtest cell or globally)
                cagr, _, _ = backtest_timed_exit(above200_opt, index_opt, buy, hold)
                results_grid.loc[buy, hold] = cagr
            except Exception as e:
                # print(f"⚠ Errore durante il backtest per {index_name} con soglia BUY={buy}% e holding={hold}d: {e}")
                results_grid.loc[buy, hold] = np.nan # Append NaN for failed backtest
 
    # Convert results_grid to numeric, coercing errors to NaN
    results_grid = results_grid.apply(pd.to_numeric, errors='coerce')
    optimization_results[index_name] = results_grid # Store results grid
 
    # Find the best combination
    if not results_grid.dropna().empty:
        best_cagr = results_grid.max().max()
        best_buy = results_grid.stack().idxmax()[0]
        best_hold = results_grid.stack().idxmax()[1]
 
        best_results_optimization[index_name] = {'Buy': best_buy, 'Holding Days': best_hold, 'CAGR': best_cagr}
 
        print(f"\n⭐ Miglior combinazione trovata per {index_name} nell'ottimizzazione:")
        print(f"   Soglia BUY: {best_buy}%")
        print(f"   Holding Days: {best_hold} giorni")
        print(f"   CAGR: {best_cagr:.4f}")
 
        # ==========================================
        # GRAFICO HEATMAP DEI RISULTATI
        # ==========================================
        plt.figure(figsize=(12, 8))
        sns.heatmap(results_grid, annot=True, fmt=".4f", cmap="viridis", cbar_kws={'label': 'CAGR'})
        plt.title(f"Heatmap Ottimizzazione Parametri per {index_name} (Soglia BUY vs Holding Days)")
        plt.xlabel("Holding Days")
        plt.ylabel("Soglia BUY (%)")
        plt.tight_layout()
        plt.show()
 
    else:
         print(f"\n❌ Nessun risultato di backtest valido trovato per {index_name} per l'ottimizzazione bidimensionale.")
 
# ==========================================
# RIEPILOGO FINALE OTTIMIZZAZIONE
# ==========================================
print(f"\n{'='*50}")
print("📊 Riepilogo Migliori Combinazioni di Parametri dall'Ottimizzazione")
print(f"{'='*50}")
 
if best_results_optimization:
    df_summary_opt = pd.DataFrame.from_dict(best_results_optimization, orient='index')
    df_summary_opt.index.name = 'Indice'
    df_summary_opt = df_summary_opt[['Buy', 'Holding Days', 'CAGR']]
    df_summary_opt.rename(columns={'Buy': 'Miglior Soglia BUY (%)'}, inplace=True)
    df_summary_opt['Miglior Soglia BUY (%)'] = df_summary_opt['Miglior Soglia BUY (%)'].astype(int)
    df_summary_opt['Holding Days'] = df_summary_opt['Holding Days'].astype(int)
    print(df_summary_opt.to_string(formatters={'CAGR': '{:.4f}'.format}))
else:
    print("❌ Nessun risultato valido dall'ottimizzazione da riepilogare.")
 
# Note: The best_results_optimization dictionary contains the results you can use
# to set the parameters in the main backtesting cell (cell 2).
 
